In [11]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import requests
import datetime as dt 
from dash.dependencies import Input, Output
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import random

from river.compose import Pipeline, FuncTransformer
from river.linear_model import LinearRegression
from river.preprocessing import StandardScaler 

from river import evaluate
from river import forest
from river import metrics

from river.utils import Rolling
from river.metrics import MAE,RMSE
from river import compose, preprocessing, linear_model

# Model

In [12]:
def get_ordinal_date(x):
    return {'closeTime': x['closeTime'].toordinal()}

In [13]:
def create_pipeline():

    pl = Pipeline(
        ('ordinal_date', FuncTransformer(get_ordinal_date)),
        ('scale', preprocessing.StandardScaler()),
        ('lr', forest.ARFRegressor(seed=42))
    )


    return pl

In [14]:
def create_metric():
    return Rolling(RMSE(), 12)

In [15]:
def learn_pred(x, y, pl, metric):

    y_pred_old = pl.predict_one(x)
    pl.learn_one(x, y)
    metric.update(y, y_pred_old)


    return x, y, y_pred_old, pl, metric

# Prediction

In [16]:
# Create model
my_pl = create_pipeline()
my_mt = create_metric()


#Document of Binance API
#https://binance-docs.github.io/apidocs/spot/en/#rolling-window-price-change-statistics

coin = 'ETHUSDT'
tick_interval = '1m'
key = "https://api.binance.com/api/v3/ticker?symbol="+coin

figure = dict(
    data=[{'x': [], 'y': []}], 
    layout=dict(
        xaxis=dict(range=[]), 
        yaxis=dict(range=[])
        )
    )

In [17]:
app = dash.Dash(__name__, update_title=None)  # remove "Updating..." from title

app.layout = html.Div(
        [
            dcc.Graph(id='graph', figure=figure), 
            dcc.Interval(id="interval", interval=1*1000)
            
        ]
    )
    
totalErr = []
predicted = 38000
rmse = 0
averge_size = 10

errList  = []
priceList= []
predictList = []
dateList = []

In [18]:
@app.callback(
    Output('graph', 'figure'), 
    [Input('interval', 'n_intervals')])
def update_data(n_intervals):
    global my_mt
    global my_pl

    print("interval ",n_intervals)

    
    data = requests.get(key).json()
    #print(data)

    price = float(data['lastPrice']) #price
    print("lastPrice = ", price)

    x = {
        'closeTime': dt.datetime.fromtimestamp(data['closeTime']/1000),
        'openPrice': float(data['openPrice']),
        'highPrice': float(data['highPrice']),
        'lowPrice': float(data['lowPrice'])
    }
    

    y = float(data['lastPrice'])

    lst_x, lst_y, lst_y_pred, my_pl, my_mt = learn_pred(x, y, my_pl, my_mt)
    print(lst_y_pred)
    if n_intervals >=300:
        print('###### RMSE :',my_mt)

    
    
    # predicted = random.randint(38000, 39000)
    predicted = lst_y_pred
    
    ''' Calculate root mean squared error 
    err = price - predicted
    square_err = err * err
    totalErr.append(square_err)
    
    #If we collect "averge_size" samples, we calculate MSE.
    if len(totalErr) == averge_size:
        mse = sum(totalErr)/averge_size
        rmse = np.sqrt(mse)
        print("rmse = ", rmse)
        errList.append(rmse)
        totalErr.clear()
    '''

    closeTime = data['closeTime']
    #print(closeTime)

    my_datetime = dt.datetime.fromtimestamp(closeTime / 1000)  # Apply fromtimestamp function
    print(type(my_datetime))             

    #return dict(x=[[my_datetime]], y=[[y_true]])
    
    dateList.append(my_datetime)
    priceList.append(price)
    predictList.append(predicted)
    
    df = pd.DataFrame(list(zip(dateList,priceList,predictList)), columns =['datetime','price','predicted'])
    
    #df['datetime']=df['datetime'].map(dt.datetime.toordinal)
    
    
    df = [
            go.Scatter(
                x=df['datetime'],
                y=df['price'],
                name='price',
                mode= 'lines+markers'
            ),
            go.Scatter(
                x=df['datetime'],
                y=df['predicted'],
                name='predicted',
                mode= 'lines+markers'
            ),
    ]

    return {'data': df } 
    
if __name__ == '__main__':
    app.run_server()

interval  None


[2024-01-01 22:45:54,197] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/dash/dash.py", line 1258, in dispatch
    ctx.run(
  F

lastPrice =  2316.44
0.0
interval  1
lastPrice =  2316.37
231.644
<class 'datetime.datetime'>


/opt/homebrew/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



interval  2
lastPrice =  2316.37
2316.4009090909094
<class 'datetime.datetime'>
interval  3
lastPrice =  2315.85
2316.3952272727274
<class 'datetime.datetime'>
interval  4
lastPrice =  2315.43
2316.2668311403504
<class 'datetime.datetime'>
interval  5
lastPrice =  2315.42
2316.095638709677
<class 'datetime.datetime'>
interval  6
lastPrice =  2315.42
2315.9394983370285
<class 'datetime.datetime'>
interval  7
lastPrice =  2315.42
2315.8639030612244
<class 'datetime.datetime'>
interval  8
lastPrice =  2315.57
2315.794845238095
<class 'datetime.datetime'>
interval  9
lastPrice =  2315.58
2315.778614123581
<class 'datetime.datetime'>
interval  10
lastPrice =  2315.69
2315.763989177489
<class 'datetime.datetime'>
interval  11
lastPrice =  2315.99
2315.7575550144297
<class 'datetime.datetime'>
interval  12
lastPrice =  2316.24
2315.776858974359
<class 'datetime.datetime'>
interval  13
lastPrice =  2316.49
2315.810051470588
<class 'datetime.datetime'>
interval  14
lastPrice =  2316.67
2315.859

[2024-01-02 07:41:51,855] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceba

interval  785


[2024-01-02 07:41:53,852] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceba

interval  786
interval  787
lastPrice =  2376.78
2373.7516216568833
###### RMSE : RMSE: 2.936322
<class 'datetime.datetime'>
lastPrice =  2376.55
2373.3942765567767
###### RMSE : RMSE: 3.014893
<class 'datetime.datetime'>
interval  788
lastPrice =  2376.77
2373.581722659872
###### RMSE : RMSE: 3.094693
<class 'datetime.datetime'>
interval  789
lastPrice =  2376.77
2373.6163104259203
###### RMSE : RMSE: 3.164216
<class 'datetime.datetime'>
interval  790
lastPrice =  2376.5
2373.8340952380954
###### RMSE : RMSE: 3.09926
<class 'datetime.datetime'>
interval  791
lastPrice =  2376.51
2373.9553221288497
###### RMSE : RMSE: 3.030943
<class 'datetime.datetime'>
interval  792
lastPrice =  2376.99
2374.055832298136
###### RMSE : RMSE: 2.996297
<class 'datetime.datetime'>
interval  793
lastPrice =  2377.0
2374.1836212121207
###### RMSE : RMSE: 2.975795
<class 'datetime.datetime'>
interval  794
lastPrice =  2377.32
2373.588499287751
###### RMSE : RMSE: 3.017147
<class 'datetime.datetime'>
interva